In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import pandas as pd
import numpy as np
from gtda.time_series import SlidingWindow
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.keras.backend import set_session
config = tf.compat.v1.ConfigProto() 
config.gpu_options.allow_growth = True  
config.log_device_placement = True  
sess2 = tf.compat.v1.Session(config=config)
set_session(sess2) 
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.layers import Activation 
from tensorflow.keras.backend import sigmoid
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import tensorflow_datasets as tfds
from tensorflow.keras.models import load_model

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:21:00.0, compute capability: 8.6



## Import Dataset

In [2]:
dataset_name = "cifar10"
(ds_train, ds_test), ds_info = tfds.load(
    dataset_name, split=["train", "test"], with_info=True, as_supervised=True
)
NUM_CLASSES = ds_info.features["label"].num_classes

IMG_SIZE = 380
batch_size = 64
size = (IMG_SIZE, IMG_SIZE)
ds_train = ds_train.map(lambda image, label: (tf.image.resize(image, size), label))
ds_test = ds_test.map(lambda image, label: (tf.image.resize(image, size), label))

def input_preprocess(image, label):
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label


ds_train = ds_train.map(
    input_preprocess, num_parallel_calls=tf.data.AUTOTUNE
)
ds_train = ds_train.batch(batch_size=batch_size, drop_remainder=True)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(input_preprocess)
ds_test = ds_test.batch(batch_size=batch_size, drop_remainder=True)

## Import Model

In [3]:
inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = inputs 
model = tf.keras.applications.EfficientNetB4(weights='imagenet',
                                             input_tensor=x,include_top=False)

model.trainable = False
x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2, name="top_dropout")(x)
outputs = layers.Dense(10, activation="softmax", name="pred")(x)
model = Model(inputs = inputs, outputs = outputs)
model.summary()

71688192/71686520 [==============================] - 3s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 380, 380, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 380, 380, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 380, 380, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 381, 381, 3)  0           normalization[0][0]              
_____________________________

## Transfer Learning on Model

In [4]:
model_name = 'enet_b4_cifar10.h5'
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(1e-2),
              metrics=['accuracy'])

checkpoint = ModelCheckpoint(model_name, save_best_only=True, monitor='val_accuracy')
model.fit(ds_train,
              epochs=10,
              validation_data=ds_test,
              callbacks=[checkpoint],
              shuffle=True,
              verbose=1)

Epoch 1/10
781/781 [==============================] - 213s 262ms/step - loss: 0.8690 - accuracy: 0.8392 - val_loss: 0.4884 - val_accuracy: 0.8807
Epoch 2/10
781/781 [==============================] - 203s 260ms/step - loss: 0.4585 - accuracy: 0.8669 - val_loss: 0.3732 - val_accuracy: 0.8829
Epoch 3/10
781/781 [==============================] - 203s 260ms/step - loss: 0.4052 - accuracy: 0.8698 - val_loss: 0.4075 - val_accuracy: 0.8724
Epoch 4/10
781/781 [==============================] - 203s 260ms/step - loss: 0.4018 - accuracy: 0.8714 - val_loss: 0.3874 - val_accuracy: 0.8784
Epoch 5/10
781/781 [==============================] - 203s 260ms/step - loss: 0.4089 - accuracy: 0.8681 - val_loss: 0.3993 - val_accuracy: 0.8736
Epoch 6/10
781/781 [==============================] - 203s 260ms/step - loss: 0.4284 - accuracy: 0.8637 - val_loss: 0.3941 - val_accuracy: 0.8753
Epoch 7/10
781/781 [==============================] - 203s 260ms/step - loss: 0.4253 - accuracy: 0.8646 - val_loss: 0.4091 -

In [5]:
def unfreeze_model(model):
    for layer in model.layers[-15:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )


unfreeze_model(model)

checkpoint = ModelCheckpoint(model_name, save_best_only=True, monitor='val_accuracy')
model.fit(ds_train,
              epochs=10,
              validation_data=ds_test,
              callbacks=[checkpoint],
              shuffle=True,
              verbose=1)

Epoch 1/10
781/781 [==============================] - 215s 268ms/step - loss: 0.3118 - accuracy: 0.8946 - val_loss: 0.2571 - val_accuracy: 0.9170
Epoch 2/10
781/781 [==============================] - 208s 267ms/step - loss: 0.2473 - accuracy: 0.9136 - val_loss: 0.2382 - val_accuracy: 0.9214
Epoch 3/10
781/781 [==============================] - 208s 267ms/step - loss: 0.2092 - accuracy: 0.9269 - val_loss: 0.2256 - val_accuracy: 0.9263
Epoch 4/10
781/781 [==============================] - 208s 267ms/step - loss: 0.1732 - accuracy: 0.9393 - val_loss: 0.2174 - val_accuracy: 0.9295
Epoch 5/10
781/781 [==============================] - 208s 267ms/step - loss: 0.1495 - accuracy: 0.9476 - val_loss: 0.2128 - val_accuracy: 0.9297
Epoch 6/10
781/781 [==============================] - 208s 267ms/step - loss: 0.1260 - accuracy: 0.9561 - val_loss: 0.2105 - val_accuracy: 0.9352
Epoch 7/10
781/781 [==============================] - 208s 267ms/step - loss: 0.1125 - accuracy: 0.9614 - val_loss: 0.2072 -

In [6]:
new_model = load_model(model_name)

In [7]:
new_model.evaluate(ds_test)

156/156 [==============================] - 35s 213ms/step - loss: 0.2075 - accuracy: 0.9353


[0.20754075050354004, 0.9352964758872986]